<a href="https://colab.research.google.com/github/RaziehAkbari2020/BigData-Spark-sql/blob/main/Terminal_with_Spark_Sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=f20680321b4250ee8c3bff2024fcfab276c9346e99cf94df984c5a34a965364c
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [98]:
from pyspark.sql import SparkSession

In [99]:
spark=SparkSession.builder.master('local[*]').appName('Busy Month').getOrCreate()

In [104]:
Psg_df =spark.read.options(sep=',', header=True, inferSchema=True).csv('/content/lax_passengers_header.csv')

In [105]:
Psg_df.show(5)

+--------------------+--------------------+-----------------+-----------------+----------------------+---------------+
|     DataExtractDate|        ReportPeriod|         Terminal|Arrival_Departure|Domestic_International|Passenger_Count|
+--------------------+--------------------+-----------------+-----------------+----------------------+---------------+
|05/01/2014 12:00:...|01/01/2006 12:00:...|Imperial Terminal|          Arrival|              Domestic|            490|
|05/01/2014 12:00:...|01/01/2006 12:00:...|Imperial Terminal|        Departure|              Domestic|            498|
|05/01/2014 12:00:...|01/01/2006 12:00:...|   Misc. Terminal|          Arrival|              Domestic|            753|
|05/01/2014 12:00:...|01/01/2006 12:00:...|   Misc. Terminal|        Departure|              Domestic|            688|
|05/01/2014 12:00:...|01/01/2006 12:00:...|       Terminal 1|          Arrival|              Domestic|         401535|
+--------------------+--------------------+-----

In [102]:
from pyspark.sql.functions import *

In [123]:
Psg_df.createOrReplaceTempView('Psg')

In [119]:
Psg_df = Psg_df.withColumn("ReportPeriod", substring(col("ReportPeriod"), 4, 7));


In [120]:
Psg_df.show(5)

+--------------------+------------+-----------------+-----------------+----------------------+---------------+
|     DataExtractDate|ReportPeriod|         Terminal|Arrival_Departure|Domestic_International|Passenger_Count|
+--------------------+------------+-----------------+-----------------+----------------------+---------------+
|05/01/2014 12:00:...|     01/2006|Imperial Terminal|          Arrival|              Domestic|            490|
|05/01/2014 12:00:...|     01/2006|Imperial Terminal|        Departure|              Domestic|            498|
|05/01/2014 12:00:...|     01/2006|   Misc. Terminal|          Arrival|              Domestic|            753|
|05/01/2014 12:00:...|     01/2006|   Misc. Terminal|        Departure|              Domestic|            688|
|05/01/2014 12:00:...|     01/2006|       Terminal 1|          Arrival|              Domestic|         401535|
+--------------------+------------+-----------------+-----------------+----------------------+---------------+
o

In [124]:
result = spark.sql("SELECT ReportPeriod AS MonthYear, SUM(Passenger_Count) AS TotalPassengers " +
                "FROM Psg " +
                "WHERE Terminal != 'Tom Bradley International Terminal' " +
                "GROUP BY ReportPeriod " +
                "HAVING SUM(Passenger_Count) > 5000000 " +
                "ORDER BY ReportPeriod");

In [125]:
result.show(5)

+---------+---------------+
|MonthYear|TotalPassengers|
+---------+---------------+
|  01/2006|       51989057|
|  01/2007|       53245682|
|  01/2008|       50684209|
|  01/2009|       47720911|
|  01/2010|       50336021|
+---------+---------------+
only showing top 5 rows

